In [ ]:
import numpy as np
import pandas as pd
import re

import requests
from bs4 import BeautifulSoup


import string
import unidecode


import transformers
import spacy

In [ ]:
DATE='2022-06-10'

MAINLINK='https://www.ejustice.just.fgov.be/cgi/summary_body.pl?language=nl&pub_date='
DETAILLINK='https://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&caller=summary&pub_date='

res = requests.get(MAINLINK + DATE)


In [ ]:

def get_numac_numbers(res:str):
    _numacs=[]

    soup = BeautifulSoup(res, 'html.parser')

    try:
        value = soup.find_all('input', {'name': 'numac'})
        
    except Exception as e:
        print("Got unhandled exception %s" % str(e))

    for v in value:
        _numacs.append(v['value'].strip())
    return _numacs



In [ ]:
def create_numac_links(_numacs:list):
    links=[]

    for _a in _numacs:
        link = f"{DETAILLINK}{DATE}&numac={_a}"
        links.append(link)
    return links

In [ ]:
numacs=get_numac_numbers(res.text)
numac_links=create_numac_links(numacs)
numac_links

In [ ]:
#Cleaning the scraped text
def clean(_a:str):
    d=re.sub(r'(?<=[.,;,:])(?=[^\s])', r' ', _a)
    
    document_test= unidecode.unidecode(d)
    document_test = document_test.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', document_test)
    # There are some instances where there is no space after '?' & ')', 
    # I am replacing these with one space so that It will not consider two words as one token.
    document_test = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    
 
    document_test = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', document_test) 
   
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    #document_test = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', document_test)

    return document_test

In [ ]:
#Scrape the article 

def scrape_numac(_numac_links:list):
    _count=0 # to check which line
    nl_list=[]
    for a in _numac_links:
        _count+=1
        res = requests.get(a)
        soup = BeautifulSoup(res.text, 'html.parser')
        for sup in soup.find_all('sup'):
            sup.unwrap()
    
        
        text=soup.text
        text=text.replace('\n',"")
        lst=text.split('Numac :')[1].split(text.split('Numac :')[2])
    

        article=lst[1].split('begin eerste woord laatste')[0].strip()
        article=clean(article)
        nl_list.append(article)
        print(_count)
    return nl_list
    

In [ ]:
nl_list=scrape_numac(numac_links)

In [ ]:
#SUMMARY

undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
    "ml6team/mbart-large-cc25-cnn-dailymail-nl-finetune"
)
tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
summarization_pipeline = transformers.pipeline(
    task="summarization",
    model=undisputed_best_model,
    tokenizer=tokenizer,
)
summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
    "nl_XX"
]

In [ ]:

#Creating the summary with pretrained model and saving to a csv file in case an error and appending to dataframe 
counter=0
def summarise(_nl_list:list):
    summary=[]
    for text in _nl_list:
        

        t= summarization_pipeline(
            text,
            do_sample=True,
            top_p=0.75,
            top_k=50,
            num_beams=4,
            min_length=50,
            early_stopping=True,
            truncation=True,
        )[0]["summary_text"]

        summary.append(t)

        counter=counter+1
        print(counter)
    return summary

In [ ]:
summary=summarise(nl_list)

### Tagging

In [ ]:
SIMILARITY=0.80
tags='aanslagjaar covid arbeidsongeschiktheidsuitkeringen bedrijfsinkomsten bedrijfskosten bedrijfstoeslag bedrijfsvoorheffing belasting belastingverdragen belastingverhoging'\
     'belastingvermindering belastingvoet belastingvoordeel belastingvrije beroepsinkomsten beroepskosten bezoldiging btw derdebetalersregeling dienstverplichtingen erfbelasting'\
     'financieringskosten heffing inkomsten inkomstenderving investeringsaftrek kapitaalaflossingen kapitaalvermindering kostenvermindering omzetbelasting personenbelasting'\
     'prestatievergoeding rechtspersonenbelasting registratierechten schenkbelasting socialezekerheidsbijdragen solidariteitsbijdrage uitbetalingsinstelling vennootschapsbelasting'\
     'verminderingen vervangingsinkomsten voorafbetalingen voorbelasting voorheffing vrijstellingsregeling waardevermindering werkgeversbijdrage werkingskosten zekerheidsbijdragen'\
     'invaliditeit verzekering'

In [ ]:
import spacy
nlp = spacy.load("nl_core_news_lg")
real_tags=nlp(tags)


In [ ]:
def tagging(real_tags,summary_tags):
    summary_tag_list={}

    for _a in summary_tags:
        
        for token in real_tags:
            q=token.similarity(_a)
        
            if q > SIMILARITY:
                
                #add token to dict
                summary_tag_list[token]=q
                #print(_a,token,_a.similarity(token))
    return summary_tag_list

In [ ]:
keys=[]
t=0
for a in summary:
    print(t)
    text=a.lower()
    summary_tags=nlp(text)      
    
    summary_tag_list=tagging(real_tags,summary_tags)
    keys.append(summary_tag_list)
    t=t+1

In [171]:
data = pd.DataFrame(
    {'date':DATE,
    'numac':numacs,
        'nllink':numac_links,
        'summary': summary,
     'nltags': keys
    })


In [172]:
import numpy as np
data['nltags']=data['nltags'].astype(str)
data['nltags']=data["nltags"].str.strip('{}')
data['nltags'] = data['nltags'].replace('',np.nan,regex = True)
data.dropna(subset = ["nltags"], inplace=True)
data

,date,numac,nllink,summary,nltags
0,2022-06-10,2022203370,https://www.ejustice.just.fgov.be/cgi/article_...,De Waalse Regering heeft een besluit tot wijzi...,"vrijstellingsregeling: 0.8031466603279114, die..."
11,2022-06-10,2022203342,https://www.ejustice.just.fgov.be/cgi/article_...,De Waalse Regering heeft een besluit tot wijzi...,"bedrijfsinkomsten: 0.8234456777572632, dienstv..."
12,2022-06-10,2022032259,https://www.ejustice.just.fgov.be/cgi/article_...,De gewijzigde bepalingen van het Personeelssta...,"dienstverplichtingen: 0.8079172968864441, vrij..."
14,2022-06-10,2022032286,https://www.ejustice.just.fgov.be/cgi/article_...,De Brusselse raad van bestuur heeft een nietig...,arbeidsongeschiktheidsuitkeringen: 0.834302306...
24,2022-06-10,2022203311,https://www.ejustice.just.fgov.be/cgi/article_...,De Grondwet is van toepassing op de werkgevers...,covid: 1.0
40,2022-06-10,2022032264,https://www.ejustice.just.fgov.be/cgi/article_...,De Brusselse regering heeft een definitief bes...,bedrijfsinkomsten: 0.8151474595069885
44,2022-06-10,2022203405,https://www.ejustice.just.fgov.be/cgi/article_...,Het ministerie van Justitie en Veiligheid heef...,"dienstverplichtingen: 0.8256614208221436, vrij..."
53,2022-06-10,2022020890,https://www.ejustice.just.fgov.be/cgi/article_...,De sociale uitkeringen voor het eerste jaar va...,arbeidsongeschiktheidsuitkeringen: 0.930212855...


In [173]:
data

,date,numac,nllink,summary,nltags
0,2022-06-10,2022203370,https://www.ejustice.just.fgov.be/cgi/article_...,De Waalse Regering heeft een besluit tot wijzi...,"vrijstellingsregeling: 0.8031466603279114, die..."
11,2022-06-10,2022203342,https://www.ejustice.just.fgov.be/cgi/article_...,De Waalse Regering heeft een besluit tot wijzi...,"bedrijfsinkomsten: 0.8234456777572632, dienstv..."
12,2022-06-10,2022032259,https://www.ejustice.just.fgov.be/cgi/article_...,De gewijzigde bepalingen van het Personeelssta...,"dienstverplichtingen: 0.8079172968864441, vrij..."
14,2022-06-10,2022032286,https://www.ejustice.just.fgov.be/cgi/article_...,De Brusselse raad van bestuur heeft een nietig...,arbeidsongeschiktheidsuitkeringen: 0.834302306...
24,2022-06-10,2022203311,https://www.ejustice.just.fgov.be/cgi/article_...,De Grondwet is van toepassing op de werkgevers...,covid: 1.0
40,2022-06-10,2022032264,https://www.ejustice.just.fgov.be/cgi/article_...,De Brusselse regering heeft een definitief bes...,bedrijfsinkomsten: 0.8151474595069885
44,2022-06-10,2022203405,https://www.ejustice.just.fgov.be/cgi/article_...,Het ministerie van Justitie en Veiligheid heef...,"dienstverplichtingen: 0.8256614208221436, vrij..."
53,2022-06-10,2022020890,https://www.ejustice.just.fgov.be/cgi/article_...,De sociale uitkeringen voor het eerste jaar va...,arbeidsongeschiktheidsuitkeringen: 0.930212855...


In [ ]:
all=list(zip(keys,summary))

In [ ]:
all